In [ ]:
!pip install -q keras
!pip install -q pandas
!pip install -q numpy
!pip install -q hyperopt
!pip install -q hyperas
!pip install -q keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
from __future__ import print_function
import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.models import  Model
from keras.layers import Dense, Input
import numpy as np
import pandas as pd

In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('drive/MyDrive/data.00.json') as user_file:
  parsed_json = user_file.read()

In [ ]:
parsed_json = parsed_json.replace('}\n{', '},{')

In [ ]:
parsed_json = '['+parsed_json+']'

In [ ]:
parsed_json[0:650]

'[{"app_name":"gendpoint","exercise_segment":"global","facility":3,"fromhost_ip":"10.7.100.7","hostname":"global-app","message":" time=\\"2019-03-19T11:00:00+01:00\\" level=info msg=\\"10.3.2.27 - global-app [19/Mar/2019:11:00:00 +0100] \\\\\\"GET /api/v1/units/status\\\\\\" 200 1452 \\\\\\"\\\\\\" \\\\\\"Go-http-client/1.1\\\\\\" (8ms)\\" clientIP=10.3.2.27 dataLength=1452 hostname=global-app latency=8 method=GET path=/api/v1/units/status referer= statusCode=200 userAgent=Go-http-client/1.1","procid":25433,"programname":"gendpoint","severity":6,"timegenerated":"2019-03-19T11:00:00.000731+01:00","timereported":"2019-03-19T11:00:00+01:00","timestamp":1552989600000},'

In [ ]:
df = pd.DataFrame(json.loads(parsed_json))

In [ ]:
del parsed_json

In [ ]:
df

,app_name,exercise_segment,facility,fromhost_ip,hostname,message,procid,programname,severity,timegenerated,timereported,timestamp
0,gendpoint,global,3,10.7.100.7,global-app,"time=""2019-03-19T11:00:00+01:00"" level=info m...",25433.0,gendpoint,6,2019-03-19T11:00:00.000731+01:00,2019-03-19T11:00:00+01:00,1552989600000
1,gendpoint,global,3,10.7.100.7,global-app,[GIN] 2019/03/19 - 11:00:00 | 200 #033[0m| ...,25433.0,gendpoint,6,2019-03-19T11:00:00.000988+01:00,2019-03-19T11:00:00+01:00,1552989600000
2,named,global,3,10.7.100.3,ns,client 4.122.55.5#58236 (12.66.66.9.in-addr.a...,4630.0,named,6,2019-03-19T11:00:00.015144+01:00,2019-03-19T11:00:00+01:00,1552989600015
3,gendpoint,global,3,10.7.100.7,global-app,"time=""2019-03-19T11:00:00+01:00"" level=info m...",25400.0,gendpoint,6,2019-03-19T11:00:00.034285+01:00,2019-03-19T11:00:00+01:00,1552989600034
4,gendpoint,global,3,10.7.100.7,global-app,[GIN] 2019/03/19 - 11:00:00 | 200 #033[0m| ...,25400.0,gendpoint,6,2019-03-19T11:00:00.034570+01:00,2019-03-19T11:00:00+01:00,1552989600034
...,...,...,...,...,...,...,...,...,...,...,...,...
1058445,gendpoint,global,3,10.7.100.7,global-app,[GIN] 2019/03/19 - 14:03:59 | 200 #033[0m| ...,25534.0,gendpoint,6,2019-03-19T14:03:59.322534+01:00,2019-03-19T14:03:59+01:00,1553000639322
1058446,gendpoint,global,3,10.7.100.7,global-app,"time=""2019-03-19T14:03:59+01:00"" level=info m...",25498.0,gendpoint,6,2019-03-19T14:03:59.330384+01:00,2019-03-19T14:03:59+01:00,1553000639330
1058447,gendpoint,global,3,10.7.100.7,global-app,[GIN] 2019/03/19 - 14:03:59 | 200 #033[0m| ...,25498.0,gendpoint,6,2019-03-19T14:03:59.330629+01:00,2019-03-19T14:03:59+01:00,1553000639330
1058448,gendpoint,global,3,10.7.100.7,global-app,"time=""2019-03-19T14:03:59+01:00"" level=info m...",25703.0,gendpoint,6,2019-03-19T14:03:59.336322+01:00,2019-03-19T14:03:59+01:00,1553000639336


In [ ]:
samples = []
for row in df.iterrows():
    index, data = row
    samples.append(data.tolist())

In [ ]:
len(samples[0])

12

In [ ]:
max_words = 200000
maxlen = 12

In [ ]:
tokenizer = Tokenizer(num_words=max_words, lower=False)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

Found 2718387 unique tokens.


In [ ]:
test_df = df[:5]

In [ ]:
test_df.drop(df.columns[[0,1, 2,3,4,5,6]], axis=1, inplace=True)

<ipython-input-31-36f33a1a9394>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(df.columns[[0,1, 2,3,4,5,6]], axis=1, inplace=True)


In [ ]:
test_df

,programname,severity,timegenerated,timereported,timestamp
0,gendpoint,6,2019-03-19T11:00:00.000731+01:00,2019-03-19T11:00:00+01:00,1552989600000
1,gendpoint,6,2019-03-19T11:00:00.000988+01:00,2019-03-19T11:00:00+01:00,1552989600000
2,named,6,2019-03-19T11:00:00.015144+01:00,2019-03-19T11:00:00+01:00,1552989600015
3,gendpoint,6,2019-03-19T11:00:00.034285+01:00,2019-03-19T11:00:00+01:00,1552989600034
4,gendpoint,6,2019-03-19T11:00:00.034570+01:00,2019-03-19T11:00:00+01:00,1552989600034


In [ ]:
t = Tokenizer()
t.fit_on_texts(test_df)
print(t)

In [ ]:
s = t.texts_to_sequences(test_df)
print(s)
w = t.word_index
print('Found %s unique tokens.' % len(word_index))
d = pad_sequences(sequences, maxlen=maxlen)

[[1], [2], [3], [4], [5]]
Found 2718387 unique tokens.


In [ ]:
len(s)

5

In [ ]:
len(sequences[0])

10

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)

In [ ]:
training_samples = int(len(samples) * 0.80)

x = data[:training_samples]
x_validation = data[training_samples:]

callbacks_list = [
    keras.callbacks.ModelCheckpoint(
    filepath='log_model-testing.h5',
    monitor='val_loss',
    save_best_only=True
    )
]


In [ ]:
input_dim = x.shape[1]
input = Input(shape=(input_dim, ))
encode = Dense(8, activation='relu')(input)
encode = Dense(4, activation='relu')(encode)

decode = Dense(8, activation='relu')(encode)
decode = Dense(12, activation='sigmoid')(decode)

autoencoder = Model(input, decode)

autoencoder.summary()

autoencoder.compile(optimizer='adam',
            loss='mean_squared_error',
            metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12)]              0         
                                                                 
 dense (Dense)               (None, 8)                 104       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 8)                 40        
                                                                 
 dense_3 (Dense)             (None, 12)                108       
                                                                 
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = autoencoder.fit(x, x,
        epochs=50,
        batch_size=32,
        callbacks=callbacks_list,
        validation_data=(x_validation, x_validation),
        shuffle=True
        )

Epoch 1/50
26462/26462 [==============================] - 109s 4ms/step - loss: 0.0021 - accuracy: 0.9190 - val_loss: 4.4600e-05 - val_accuracy: 0.9699
Epoch 2/50
26462/26462 [==============================] - 110s 4ms/step - loss: 1.2912e-04 - accuracy: 0.9451 - val_loss: 3.4081e-05 - val_accuracy: 0.9776
Epoch 3/50
26462/26462 [==============================] - 101s 4ms/step - loss: 9.9170e-05 - accuracy: 0.9482 - val_loss: 2.9046e-05 - val_accuracy: 0.9757
Epoch 4/50
26462/26462 [==============================] - 107s 4ms/step - loss: 7.4581e-05 - accuracy: 0.9495 - val_loss: 2.3518e-05 - val_accuracy: 0.9770
Epoch 5/50
26462/26462 [==============================] - 108s 4ms/step - loss: 6.1498e-05 - accuracy: 0.9454 - val_loss: 1.9763e-05 - val_accuracy: 0.9754
Epoch 6/50
26462/26462 [==============================] - 110s 4ms/step - loss: 5.3818e-05 - accuracy: 0.9437 - val_loss: 2.0745e-05 - val_accuracy: 0.9761
Epoch 7/50
26462/26462 [==============================] - 101s 4ms/s

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from numpy import vstack

# Combined x and x_test.

predictions = autoencoder.predict(data)

# Calculated MSE.
mse = np.mean(np.power(data - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse})

# Set threshold at the 99th quartile.
threshold = error_df.quantile(0.999)

error_df.describe()

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.plot(error_df.index, error_df.reconstruction_error, marker='o', ms=3.5, linestyle='')
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='threshold')
ax.legend(loc=5)
plt.title("Reconstruction error")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [ ]:
index_list = []
for index, row in error_df.iterrows():
    if row.reconstruction_error > np.float(threshold):
        index_list.append(index)


In [ ]:
len(index_list)

In [ ]:
anomaly = []
for index_val in index_list:
  anomaly.append(df.iloc[index_val])

In [ ]:
anomaly